# Data Wrangling with MongoDB

### Project Overview

In this project, I will take a section of the open street map dataset and perform some data cleaning.

### Map Area

Denver/Boulder metro, CO

### Objectives

1. Convert xml data to json
2. Cleanup street names and zip codes
3. Move cleaned dataset to a mongoDB instance

### Data Import

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections
import pymongo
from collections import defaultdict

### Creating a Sample Dataset from the Denver/Boulder Dataset

Because the Denver/Boulder dataset is pretty large, I want to use a smaller section of it while I'm testing code. 

In [2]:
OSM_FILE = "denver-boulder_colorado.osm"  
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [3]:
# Here I want to get a general idea of the scope of the dataset
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
den_tags = count_tags(SAMPLE_FILE)
pprint.pprint(den_tags)

{'member': 3714,
 'nd': 391253,
 'node': 336500,
 'osm': 1,
 'relation': 164,
 'tag': 187336,
 'way': 37141}


In [5]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map(SAMPLE_FILE)
pprint.pprint(keys)

{'lower': 111809, 'lower_colon': 71853, 'other': 3674, 'problemchars': 0}


In [6]:
#people invovlved in the map editing.
def unique_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = unique_users(SAMPLE_FILE)
len(users)

1240

### Investigating Street Names

Next, I need to get a better understanding of all of the different street names used in this dataset, to see if there are inconsistencies that need to be fixed.

In [7]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'street' :"Street",
           'st'   : "Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square",
           'Raod' : "Road",}

            


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

In [8]:
street_types = audit(SAMPLE_FILE)
pprint.pprint(dict(street_types))

{'119': set(['Highway 119']),
 '19': set(['South Rock Creek Parkway, Building 19']),
 '2': set(['Colorado SH 2', 'Colorado SR 2']),
 '2200': set(['18th Street Suite 2200']),
 '285': set(['Highway 285']),
 '287': set(['US Highway 287']),
 '52': set(['I-76 & CO Hwy 52']),
 '72': set(['CO 72']),
 '73': set(['Hwy 73']),
 '74': set(['Highway 74']),
 '83': set(['State Highway 83']),
 '900': set(['West 120th Avenue, Suite 900']),
 'Ave': set(['E Caley Ave',
             'E Fair Ave',
             'E Maplewood Ave',
             'W Plymouth Ave',
             'W. 10th Ave',
             'Washington Ave']),
 'Avenue)': set(['East Bromley Lane (152nd Avenue)']),
 'Baselin': set(['Baselin']),
 'Baseline': set(['Baseline']),
 'Bikeway': set(['US 36 Bikeway']),
 'Blvd': set(['Colorado Blvd',
              'Green Valley Ranch Blvd',
              'S University  Blvd',
              'Sheridan Blvd',
              'Wadsworth Blvd']),
 'Broadway': set(['Broadway', 'S Broadway', 'South Broadway']),
 'Ca

In [35]:
def update_street(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)
    return name

for street_type, ways in street_types.iteritems():
    for name in ways:
        new_name = update_street(name, mapping, street_type_re)
        print name, "=>", new_name

East Colfax => East Colfax
Baseline => Baseline
Caria => Caria
South Carr Avenue West => South Carr Avenue West
Inverness Drive West => Inverness Drive West
Park Avenue West => Park Avenue West
Sterling Circle West => Sterling Circle West
Main => Main
Via Varra => Via Varra
E. Rim Rd => E. Rim Road
E Arapahoe Rd => E Arapahoe Road
Pine Valley Rd => Pine Valley Road
Newland => Newland
S Niagra Ct => S Niagra Court
South 57th Ct => South 57th Court
Devonshire Way => Devonshire Way
East 60th Way => East 60th Way
South Xapary Way => South Xapary Way
South Fultondale Way => South Fultondale Way
Justice Way => Justice Way
Airport Way => Airport Way
S Monaco Way => S Monaco Way
Golden Eagle Way => Golden Eagle Way
South Grand Baker Way => South Grand Baker Way
Reed Way => Reed Way
South Dayton Way => South Dayton Way
Wisteria Way => Wisteria Way
Aberdeen Way => Aberdeen Way
West 68th Way => West 68th Way
Kincross Way => Kincross Way
Progress Way => Progress Way
Meredith Way => Meredith Way
We

In [20]:
zip_code_range = [80000,81000]
zip_code_default = 80203

def audit_zipcode(invalid_zipcodes, zipcode):
    try:
        if not (zip_code_range[0] <= int(zipcode) <= zip_code_range[1]):
            raise ValueError
    except ValueError:
        invalid_zipcodes[zipcode] += 1
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

invalid_zipcode = audit_zip(SAMPLE_FILE)
pprint.pprint(dict(invalid_zipcode))

{'1800': 1,
 '80010-1425': 1,
 '801112': 1,
 '80214-1803': 1,
 '80214-1833': 1,
 '80219-1535': 1,
 '80226-2975': 1,
 '80229-7923': 1,
 '80303-1229': 1,
 '80503-7570': 1,
 '80504-6311': 1,
 '80504-8601': 1,
 'CO': 1,
 'CO 80211': 1,
 'CO 80305': 1,
 'CO 80439': 1,
 'Golden, CO 80401': 1}


In [38]:
def update_zip(zipcode):
    try:
        if not (zip_code_range[0] <= int(zipcode) <= zip_code_range[1]):
            raise ValueError
        else:
            return int(zipcode)
    except ValueError:
        return zip_code_default

In [40]:

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(e):
    node = {}
    node['created'] = {}
    node['pos'] = [0,0]
    if e.tag == "way":
        node['node_refs'] = []
    if e.tag == "node" or e.tag == "way" :
        node['type'] = e.tag
        #attributes
        for k, v in e.attrib.iteritems():
            #latitude
            if k == 'lat':
                try:
                    lat = float(v)
                    node['pos'][0] = lat
                except ValueError:
                    pass
            #longitude
            elif k == 'lon':
                try:
                    lon = float(v)
                    node['pos'][1] = lon
                except ValueError:
                    pass
            #creation metadata
            elif k in CREATED:
                node['created'][k] = v
            else:
                node[k] = v
        #children
        for tag in e.iter('tag'):
            k = tag.attrib['k']
            v = tag.attrib['v']
            if problemchars.match(k):
                continue
            elif lower_colon.match(k):
                k_split = k.split(':')
                #address fields
                if k_split[0] == 'addr':
                    k_item = k_split[1]
                    if 'address' not in node:
                        node['address'] = {}
                    #streets
                    if k_item == 'street':
                        v = update_street(v, mapping, street_type_re)                    
                    #postal codes
                    if k_item == 'postcode':
                        v = update_zip(v)
                    node['address'][k_item] = v
                    continue
            node[k] = v
        #way children
        if e.tag == "way":
            for n in e.iter('nd'):
                ref = n.attrib['ref']
                node['node_refs'].append(ref);
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


In [41]:
'''
PROCESS DATA AND OUTPUT JSON
'''
process_map(SAMPLE_FILE, False)

[{'created': {'changeset': '7551724',
   'timestamp': '2011-03-14T04:12:27Z',
   'uid': '117055',
   'user': 'GPS_dr',
   'version': '6'},
  'id': '25676629',
  'pos': [39.9822661, -105.2638756],
  'type': 'node'},
 {'created': {'changeset': '209539',
   'timestamp': '2007-02-06T20:35:37Z',
   'uid': '5879',
   'user': 'erinn',
   'version': '1'},
  'id': '25689364',
  'pos': [39.9816945, -105.261765],
  'type': 'node'},
 {'created': {'changeset': '15013106',
   'timestamp': '2013-02-13T00:22:23Z',
   'uid': '1209932',
   'user': 'oddityoverseer',
   'version': '3'},
  'id': '25696400',
  'pos': [40.0035047, -105.2624665],
  'type': 'node'},
 {'created': {'changeset': '1918371',
   'timestamp': '2009-07-23T19:31:25Z',
   'uid': '1679',
   'user': 'andrewpmk',
   'version': '2'},
  'id': '25757811',
  'pos': [40.0073148, -105.2534844],
  'type': 'node'},
 {'created': {'changeset': '27947599',
   'timestamp': '2015-01-06T04:34:31Z',
   'uid': '2237750',
   'user': 'chachafish',
   'versi

In [42]:
data[0]

NameError: name 'data' is not defined